In [1]:
! pip install cohere pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 950.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.0/211.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 20.1 MB/s eta 0:00:00
  Attempting uninstall: importlib_metadata
    Found existing installation: importlib-metadata 7.0.1
    Uninstalling importlib-metadata-7.0.1:
      Successfully uninstalled importlib-metadata-7.0.1


In [54]:
cohere_api_key = ''
pinecone_api_key = ''

In [3]:
import cohere
co = cohere.Client(cohere_api_key)

In [4]:
from pinecone import Pinecone

pc = Pinecone(pinecone_api_key) # add your pinecone API key here

index_name = 'quranic'

# if the index does not exist, we create it
if index_name not in pc.list_indexes().names():
    pc.create_index(
        index_name,
        dimension=shape[1],
        metric='cosine'
    )

# connect to index
index = pc.Index(index_name)

In [55]:
limit = 1600

def retrieve(query):
    xq = co.embed(
        texts=[query],
        model='multilingual-22-12',
        truncate='NONE'
    ).embeddings
    # search pinecone index for context passage with the answer
    xc = index.query(vector=xq, top_k=3, include_metadata=True)

    # Extract relevant information from the matches
    surahs = [str(x['metadata']['Surah']) for x in xc['matches']]
    ayahs = [str(x['metadata']['Ayat']) for x in xc['matches']]
    arabics = [str(x['metadata']['Arabic']) for x in xc['matches']]
    tafaseers = [str(x['metadata']['Tafaseer1']) for x in xc['matches']]

    # Combine the information into formatted contexts
    contexts = [
        f"Surah: {surah}\n Ayah: {ayah}\n Arabic: {arabic}\n"
        for surah, ayah, arabic in zip(surahs, ayahs, arabics)
    ]

    # Build the prompt with the retrieved contexts included
    prompt_start = (
        f"Answer the Query based on the contexts, if it's not in the contexts say 'I don't know the answer'. \n\n"
        f"Context:\n"
    )
    prompt_end = (
        f"\n\nQuery: {query}\nAnswer in the language of Query, if Query is in English Answer in English. Please provide reference Quran verses."
    )

    # Append contexts until hitting the limit
    for i in range(1, len(contexts)):
        if len("".join(contexts[:i])) >= limit:
            prompt = (
                prompt_start +
                "".join(contexts[:i-1]) +
                prompt_end
            )
            break
        elif i == len(contexts)-1:
            prompt = (
                prompt_start +
                "".join(contexts) +
                prompt_end
            )
    return prompt

# Example usage with a query in a different language (Spanish)
query_new = "¿Cuál es el significado de la vida?"
prompt_new = retrieve(query_new)
print(prompt_new)

Answer the Query based on the contexts, if it's not in the contexts say 'I don't know the answer'. 

Context:
Surah: 29.0
 Ayah: 64.0
 Arabic: وَمَا هَـٰذِهِ ٱلْحَيَاةُ ٱلدُّنْيَآ إِلاَّ لَهْوٌ وَلَعِبٌ وَإِنَّ ٱلدَّارَ ٱلآخِرَةَ لَهِيَ ٱلْحَيَوَانُ لَوْ كَانُواْ يَعْلَمُونَ
Surah: 40.0
 Ayah: 39.0
 Arabic: يٰقَوْمِ إِنَّمَا هَـٰذِهِ ٱلْحَيَاةُ ٱلدُّنْيَا مَتَاعٌ وَإِنَّ ٱلآخِرَةَ هِيَ دَارُ ٱلْقَـرَارِ
Surah: 79.0
 Ayah: 38.0
 Arabic: وَآثَرَ ٱلْحَيَاةَ ٱلدُّنْيَا


Query: ¿Cuál es el significado de la vida?
Answer in the language of Query, if Query is in English Answer in English. Please provide reference Quran verses.


In [56]:
def complete(prompt):
  response = co.generate(
                          model='c4ai-aya',
                          prompt=prompt,
                          max_tokens=600,
                          temperature=0.2,
                          k=0,
                          stop_sequences=['\n\n'],
                          return_likelihoods='NONE'
                        )
  return response.generations[0].text.strip()

In [57]:
complete(prompt_new)

'The life of this world is but a play and a game, and the hereafter is the real life. This is stated in the Quran, "And this life of the world is naught but a vain thing and a play, and the hereafter is better for those who fear Allah." (Quran 40:39)'